# Surprise 파헤치기

Scikit-Surprise는 추천시스템을 다루는 라이브러리이며, Scikit-Learn에서 많은 머신러닝 모델을 다뤘던 것 처럼, Trainset과 testset으로 나누고, validation을 통해 RMSE등과 같은 평가 지표를 계산해볼 수 있다. 하지만 '추천시스템'이라는 특성상 기존의 머신러닝과 구조가 조금 다른 데이터를 다루게된다.

More Detail --> https://github.com/NicolasHug/Surprise

## TrainSet Object

Surprise에는 __TrainSet__이라는 객체가 있다. 이를 생성하기 위해서는 기본 데이터가 있어야하는데, 익히 알고있는 DataFrame형태를 다룬다고 가정을 한다. 물론 다른 형태의 파일을 불러올 수도 있다.

reader = Reader(line_format='user item rating')  
data = Dataset.load_from_df(df=df, reader=reader)  
trainSet = data.build_full_trainset()  

load_from_df 함수안에 df, reader를 인자로 넘겨주어야 하며, DataFrame에서 Column의 순서는 User, Item, Rating을 지켜주어야 하는 것이다. data변수 안에는 raw_ratings가 저장되어있는데, (userID, itemID, rating)의 튜플이 리스트안에 저장되어있는 형태이다. __build_full_trainSet()__함수를 쓰게 되면 내부적으로 raw_rating에 construct_train_set함수를 실행시켜 TrainSet객체를 생성하게 된다. 

TrainSet 객체 안에는 __ur, ir, n_users, n_items, n_ratings, rating_scale, raw2inner_id_users, raw2inner_id_items__ 등이 저장되어 있는데 raw한 userID, itemID를 numerical(0 ~ n)하게 재구성한 것이다. 이렇게 한 이유는 나중에 similarity matrix를 만들 때 index에 접근을 용이하게 하기위함이다. 자연스럽게 raw한 id를 numerical한 inner id로 바꾸는 기능을 하는 두가지 함수도 존재하게 된다.

Surprise에서 어떠한 모델에 학습시킬 때는 항상 이 TrainSet객체를 학습시키게 되며, 학습시킨다는 것은 그 모델에 해당하는 __Similarity Matrix를 만드는 것__ 이라고 이해를 하면 된다. 기존에 Scikit-Learn에서 학습을 시킬 때는 DataFrame의 형태를 학습시켜주었지만, Surprise에서는 이러한 차이점이 있다.

TrainSet Object에서는 __build_testset()__, __build_anti_testset()__와 같은 함수를 사용할 수 있는데 전자는 모든 rating이 담긴 dataframe (u, i, r)을 raw한 형태로 재구성한 것이고, build_anti_testset()은 유저가 보지않은 것들만 모아서 정리를 해둔 dataframe이다. trainSet에서 모든 영화의 갯수를 알 수 있을 것이고, 그에 따라 유저별로 보지 않은 영화를 파악할 수 있을 것이다. 보지 않았기 때문에 rating값이 존재하지 않게 되는데, __fill__이라는 인자를 넘겨주어 모두 같은 값으로 채우게 된다. fill값을 따로 지정해주지 않게 되면 모든 rating의 평균값인 global_mean값을 사용한다.

## train_test_split function

Surprise에서도 train과 test를 나누어서 학습시키고, testset을 통해서 모델을 평가하는 방식을 취한다.

self.trainSet, self.testSet = train_test_split(data, test_size=.25, random_state=1)

train_test_split안에는 TrainSet객체가 아닌 load_from_df에서 불러들인 data를 인자로 넘겨주게 되며, 리스트안에 튜플 형태의 rating들을 비율에 맞춰 쪼갠다. trainset에 해당하는 부분은 TrainSet Object형태로 다시 변환하여 저장하게 되며, testset에 해당하는 부분은 raw한 형태 그대로 저장한다. Surprise에서 학습을 시키는 trainset은 항상 TrainSet object형태 이며, 학습된 모델에 평가나 예측을 위해서 넣는 testdata는 raw한 형태임을 기억하자. 모델의 학습은 similarity matrix를 만드는 것이며 matrix의 index접근을 용이하게 하기위해서 TrainSet object객체가 필요하다.

## LOOCVTrain, LOOCVTest

추천시스템에는 평가지표가 엄청나게 많이 존재하는데, 그중 하나가 Hit-Rate이라는 것이다. Hit-Rate은 유저가 본 영화들 중에서 하나를 뽑아 안봤다고 가정한 뒤에 모델에 학습을 시키고, TOP N을 뽑는다. 그 TOP N 속에 제외시켜두었던 영화가 존재하면, 추천시스템이 의미가 있다고 보는 것이 Hit-Rate이다. 제외시켜두었던 영화는 특정유저가 이미 본 영화이기 때문에, 관심이 있는 영화라고 간주할 수 있는 것이다. 그 영화의 rating이 낮을 수도 있겠지만, Hit-Rate관점에서는 그 영화의 rating이 어떻든 간에 보기만 하였다면 관심이 있는 것으로 간주한다. 이렇게 Hit-Rate를 구하기 위해서 필요한 data가 바로 LOOCVTrain, LOOCVTest이다. 단순 예측을 위해서라면 이러한 데이터가 쓸모가 없다.

LOOCVTrain, LOOCVTest를 만들때는 Raw한 데이터를 활용하게 되며, 유저별로 random하게 하나의 영화를 뽑아서 LOOCVTest에 포함시켜놓고, LOOCVTrain에는 하나씩 제외된 데이터가 저장되게 된다. LOOCVTrain은 역시 TrainSet Object형태로 저장이되고, LOOCVTest는 raw한 형태 그대로 저장된다. 이때, 모든 유저 대상으로 하는 것이 아니라, min_n_ratings를 설정하여 최소 몇개의 rating을 남긴 유저만 대상으로 선택할 수 있다.

LOOCVTrain 또한 TrainSet객체이기 때문에 위에서 사용한 build_testset, build_anti_testset함수를 사용할 수 있게 된다. Hit Rate를 구할 때 Top N을 구하는 방법은 두가지가 있다. prediction을 하는 방법과 하지않는 방법인데, prdiction을 하지 않는다면 anti_testset을 이용하지 않는다. similarity matrix를 활용하여 모든 아이템에 대한 rating을 간단하게 구한 다음에 top N만 추리는 형식이다. 이때 본 것을 필터링 하는 방식으로 안본 item을 추천해주게 된다. prediction을 하는 방법은 GetLOOCVTrainSet을 학습시킨후, LOOCVAntiTrainSet을 예측하여 유저가 보지 않은 아이템에 대해서만 예측을 해준다. AntiTrainSet을 만들때 fill로 모든 rating을 같은 값으로 채워줬었는데, similarity에 의해서 rating이 달라지게 된다. rating을 정렬하여 top N를 뽑은 다음, LOOCVTestSet과 비교하여 유저별로 그 아이템이 TopN에 포함되어있는지 평가하게 된다. 그렇기 때문에 Top N을 시행하는 함수는 모든 유저에 대한 TopN을 가지고 있도록 해야한다.